In [1]:
import numpy as np
from scipy.stats import t
from scipy.optimize import minimize

### 1.
Use the Newton's method to find the root of $f(x)=x^3-9$.

In [2]:
x = 100
for i in range(100):
    f = x**3 - 9
    fprime = 3*x**2
    x = x - f/fprime

print('The root of given equation is', x)

The root of given equation is 2.080083823051904


### 2.
(a) Generate independent random sample $X_1, X_2,\dots, X_n$ from $N(\mu, \sigma^2)$ of size $n=100$ with $\mu=5, \sigma^2=9$, and the find 95% confidence interval for $\mu$ using t-distribution.

(b) Repeat $N=500$ times Monte-Carlo experiment for (a) to find empirical coverage and average length of the 95% confidence intervals, simultaneously in one function of python coding.

(c) For the sample of (a) above, find i.i.d bootstrap sample and compute bootstrap sample mean.

(d) Repeat the bootstrap procedure $B=500$ times for (c) to obtain 95% bootstrap confidence interval for $\mu$.

**(a)**

In [3]:
data = np.random.normal(loc=5, scale=3, size=100)
X_bar = data.mean()
sigma = data.std()
lower = X_bar - t.ppf(q=0.975, df=99)*(sigma/np.sqrt(len(data)))
upper = X_bar + t.ppf(q=0.975, df=99)*(sigma/np.sqrt(len(data)))
print(f'95% confidence interval for mu: [{lower:.2f}, {upper:.2f}]')

95% confidence interval for mu: [4.77, 5.97]


**(b)**

In [4]:
def Montecarlo(mu, s, alpha, n, N):
    cover = 0
    length = []
    for i in range(N):
        data = np.random.normal(loc=mu, scale=s, size=n)
        X_bar = data.mean()
        simga = data.std()
        lower = X_bar - t.ppf(q=1-alpha/2, df=99)*(sigma/np.sqrt(n))
        upper = X_bar + t.ppf(q=1-alpha/2, df=99)*(simga/np.sqrt(n))

        length.append(upper - lower)
        if lower < 5 and upper > 5:
            cover += 1
    
    empirical_coverage = cover/N
    average_length = np.mean(length)

    return empirical_coverage, average_length

In [5]:
Montecarlo(5, 3, 0.05, 100, 500)

(0.97, 1.1922105377237808)

**(c)**

In [6]:
bootstrap = np.random.choice(data, size=100, replace=True)  # data in 2.(a)
bootstrap.mean()

4.964533148129266

**(d)**

In [7]:
boot_means = []
for i in range(500):
    bootstrap = np.random.choice(data, size=100, replace=True)  # data in 2.(a)
    boot_means.append(bootstrap.mean())
    
boot_means.sort()
lower = boot_means[int(500*(0.05/2))]
upper = boot_means[int(500*(1 - 0.05/2))]
print(f'95% bootstrap confidence interval for mu: [{lower:.2f}, {upper:.2f}]')

95% bootstrap confidence interval for mu: [4.79, 5.96]


### 3.
(a) Generate dependent random sample $X_1, X_2, \dots, X_n$ of size $n=400$ satisfying
$$X_t = 0.3 + 0.5X_{t-1} + \epsilon_t,\ t=1,2,\dots,n\ with\ X_0=0$$
where $\epsilon_1, \epsilon_2, \dots, \epsilon_n \sim N(0,1)$. Find its mean.

(b) Generate moving block bootstrap sample from (a) with block length $l=20, 40$ and find the means. Among the cases of $l=20, 40$, which is closer to the mean in (a)?

(c) Generate stationary bootstrap sample from (a) with geometric parameter $p=0.05, 0.5$ and find means. Among the cases of $p=0.05, 0.5$, which is closer to the mean in (a)?

**(a)**

In [8]:
X = 0
X_list = []
for i in range(400):
    X = 0.3 + 0.5*X + np.random.randn()
    X_list.append(X)

np.mean(X_list)

0.4066032356113061

**(b)**

In [9]:
def moving_block_boot_sample(data, length):
    blocks = int(len(data) / length)
    start = np.random.choice(len(data)-length, size=blocks, replace=True)
    sample = []
    
    for i in range(blocks):
        sample += data[start[i]:start[i]+20]
    
    return np.mean(sample)

In [10]:
mean_20 = moving_block_boot_sample(X_list, 20)  # X_list in 3.(a)
mean_40 = moving_block_boot_sample(X_list, 40)

print(f'real_mean = {np.mean(X_list):.4f}')
print(f'mean_20 = {mean_20:.4f}')
print(f'mean_40 = {mean_40:.4f}')
print('The case of l= is more closer to the real mean.')

real_mean = 0.4066
mean_20 = 0.4566
mean_40 = 0.2982
The case of l= is more closer to the real mean.


**(c)**

In [11]:
def stationary_boot_sample(data, p):
    N = 0
    sample = []
    
    while N < len(data):
        length = np.random.geometric(p)
        if length >= len(data):
            length = len(data) - 1
        
        start = np.random.choice(len(data)-length)
        sample += data[start:start+length]
        N += length
        
    sample = sample[:len(data)]
    return np.mean(sample)

In [12]:
mean_05 = stationary_boot_sample(X_list, 0.05)  # X_list in 3.(a)
mean_5 = stationary_boot_sample(X_list, 0.5)

print(f'real_mean = {np.mean(X_list):.4f}')
print(f'mean_05 = {mean_05:.4f}')
print(f'mean_5 = {mean_5:.4f}')
print('The case of p= is more closer to the real mean.')

real_mean = 0.4066
mean_05 = 0.4829
mean_5 = 0.3333
The case of p= is more closer to the real mean.


### 4.
For the data in problem 3(a) above, we suppose that $a_0=0.3$ and $a_1=0.5$ are unknown in the regression $X_t = a_0 + a_1 X_{t-1} + \epsilon_t$. Find estimates of the two parameters $(a_0, a_1)$ using the maximum likelihood estimates. Are your answers similar to the true values?

In [13]:
x = np.array([0] + X_list[:len(X_list)-1])  # X_{t-1}: X_0, ..., X_399
y = X_list  # X_{t}: X_1, ..., X_400
n = len(x)

def log_likelihood(parameters):
    yhat = parameters[0] + parameters[1]*x
    res = y - yhat
    LL = -n/2*np.log(2*np.pi) - n*np.log(sigma) - 1/(2*sigma**2)*np.sum((res)**2)
    return -LL

In [14]:
parameters = [0, 0, 1]
estimates = minimize(log_likelihood, parameters).x
print(f'a0 = {estimates[0]}, a1 = {estimates[1]}')
print('MLE is similar to the true values(0.3, 0.5).')

a0 = 0.21559593395467097, a1 = 0.47223054928978614
MLE is similar to the true values(0.3, 0.5).
